In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, recall_score

In [2]:
consdf = pd.read_parquet('/uss/hdsi-prismdata/q2-ucsd-consDF.pqt')
consdf

,prism_consumer_id,evaluation_date,credit_score,DQ_TARGET
0,0,2021-09-01,726.0,0.0
1,1,2021-07-01,626.0,0.0
2,2,2021-05-01,680.0,0.0
3,3,2021-03-01,734.0,0.0
4,4,2021-10-01,676.0,0.0
...,...,...,...,...
14995,14995,2022-03-08,655.0,NaN
14996,14996,2022-01-15,625.0,NaN
14997,14997,2022-01-31,688.0,NaN
14998,14998,2022-03-08,722.0,NaN


In [3]:
acctdf = pd.read_parquet('/uss/hdsi-prismdata/q2-ucsd-acctDF.pqt')
acctdf

,prism_consumer_id,prism_account_id,account_type,balance_date,balance
0,3023,0,SAVINGS,2021-08-31,90.57
1,3023,1,CHECKING,2021-08-31,225.95
2,4416,2,SAVINGS,2022-03-31,15157.17
3,4416,3,CHECKING,2022-03-31,66.42
4,4227,4,CHECKING,2021-07-31,7042.90
...,...,...,...,...,...
24461,11500,24461,CHECKING,2022-03-27,732.75
24462,11615,24462,SAVINGS,2022-03-30,5.00
24463,11615,24463,CHECKING,2022-03-30,1956.46
24464,12210,24464,CHECKING,2022-03-28,2701.51


In [4]:
trxndf = pd.read_parquet('/uss/hdsi-prismdata/q2-ucsd-trxnDF.pqt')
trxndf

,prism_consumer_id,prism_transaction_id,category,amount,credit_or_debit,posted_date
0,3023,0,4,0.05,CREDIT,2021-04-16
1,3023,1,12,481.56,CREDIT,2021-04-30
2,3023,2,4,0.05,CREDIT,2021-05-16
3,3023,3,4,0.07,CREDIT,2021-06-16
4,3023,4,4,0.06,CREDIT,2021-07-16
...,...,...,...,...,...,...
6407316,10533,6405304,31,4.96,DEBIT,2022-03-11
6407317,10533,6405305,12,63.48,DEBIT,2022-03-30
6407318,10533,6405306,12,53.99,DEBIT,2022-03-30
6407319,10533,6405307,12,175.98,DEBIT,2022-03-31


### week 1 - feature importance

looking at how often balances get below $0 because it directly measures behavior and failures states 

In [5]:
acctdf['balance_date'] = pd.to_datetime(acctdf['balance_date'])


In [6]:
df = acctdf.merge(consdf, on='prism_consumer_id')
df

,prism_consumer_id,prism_account_id,account_type,balance_date,balance,evaluation_date,credit_score,DQ_TARGET
0,3023,0,SAVINGS,2021-08-31,90.57,2021-09-01,699.0,0.0
1,3023,1,CHECKING,2021-08-31,225.95,2021-09-01,699.0,0.0
2,4416,2,SAVINGS,2022-03-31,15157.17,2021-04-01,724.0,NaN
3,4416,3,CHECKING,2022-03-31,66.42,2021-04-01,724.0,NaN
4,4227,4,CHECKING,2021-07-31,7042.90,2021-08-01,707.0,NaN
...,...,...,...,...,...,...,...,...
24461,11500,24461,CHECKING,2022-03-27,732.75,2022-03-27,686.0,0.0
24462,11615,24462,SAVINGS,2022-03-30,5.00,2022-03-30,664.0,0.0
24463,11615,24463,CHECKING,2022-03-30,1956.46,2022-03-30,664.0,0.0
24464,12210,24464,CHECKING,2022-03-28,2701.51,2022-03-28,660.0,0.0


In [7]:
df['DQ_TARGET'].value_counts()

DQ_TARGET
0.0    17979
1.0     1508
Name: count, dtype: int64

In [8]:
DQ_0 = df[df['DQ_TARGET'] == 0.0]
DQ_0

,prism_consumer_id,prism_account_id,account_type,balance_date,balance,evaluation_date,credit_score,DQ_TARGET
0,3023,0,SAVINGS,2021-08-31,90.57,2021-09-01,699.0,0.0
1,3023,1,CHECKING,2021-08-31,225.95,2021-09-01,699.0,0.0
5,3920,5,SAVINGS,2021-10-31,0.26,2021-11-01,736.0,0.0
6,3920,6,CHECKING,2021-10-31,4.42,2021-11-01,736.0,0.0
7,3920,7,CHECKING,2021-10-31,1.05,2021-11-01,736.0,0.0
...,...,...,...,...,...,...,...,...
24461,11500,24461,CHECKING,2022-03-27,732.75,2022-03-27,686.0,0.0
24462,11615,24462,SAVINGS,2022-03-30,5.00,2022-03-30,664.0,0.0
24463,11615,24463,CHECKING,2022-03-30,1956.46,2022-03-30,664.0,0.0
24464,12210,24464,CHECKING,2022-03-28,2701.51,2022-03-28,660.0,0.0


In [9]:
DQ_1 = df[df['DQ_TARGET'] == 1.0 ]
DQ_1

,prism_consumer_id,prism_account_id,account_type,balance_date,balance,evaluation_date,credit_score,DQ_TARGET
29,196,29,SAVINGS,2021-08-27,18.86,2021-09-01,573.0,1.0
30,196,30,CHECKING,2021-11-30,1282.98,2021-09-01,573.0,1.0
55,3806,55,SAVINGS,2021-09-30,0.00,2021-10-01,680.0,1.0
56,3806,56,CHECKING,2021-09-30,4523.18,2021-10-01,680.0,1.0
239,2162,239,SAVINGS,2021-02-28,193.25,2021-03-01,665.0,1.0
...,...,...,...,...,...,...,...,...
24230,12880,24230,SAVINGS,2022-03-27,8044.50,2022-03-27,664.0,1.0
24233,12880,24233,CHECKING,2022-03-27,491.59,2022-03-27,664.0,1.0
24399,12162,24399,CHECKING,2021-12-09,154.58,2021-12-09,654.0,1.0
24424,12162,24424,CHECKING,2021-12-09,126.27,2021-12-09,654.0,1.0


In [10]:
DQ_1['credit_score'].mean()

593.8415119363395

In [11]:
df['credit_score'].mean()

665.3855963377749

In [12]:
df['balance'].mean()

4296.199248753373

In [13]:
DQ_1['balance'].mean()

1284.024675066313

In [14]:
df[df['balance'] < 0]

,prism_consumer_id,prism_account_id,account_type,balance_date,balance,evaluation_date,credit_score,DQ_TARGET
82,2246,82,CHECKING,2021-04-30,-168.90,2021-05-01,703.0,0.0
206,2751,206,CHECKING,2022-03-30,-474.41,2022-04-01,677.0,0.0
506,2242,506,CHECKING,2022-03-31,-49.52,2022-02-01,662.0,0.0
616,3280,616,CHECKING,2021-11-30,-7.23,2021-12-01,680.0,0.0
904,1056,904,CHECKING,2021-03-31,-313.61,2021-04-01,655.0,0.0
...,...,...,...,...,...,...,...,...
24239,12997,24239,CREDIT CARD,2021-12-14,-25.00,2021-12-14,695.0,0.0
24279,13976,24279,CREDIT CARD,2022-02-15,-1025.27,2022-02-15,806.0,0.0
24296,11678,24296,CHECKING,2021-12-19,-145.00,2021-12-19,657.0,0.0
24357,10353,24357,SAVINGS,2021-12-16,-10.00,2021-12-16,685.0,0.0


In [15]:
balance_p10 = (
    df.groupby('prism_consumer_id')['balance']
      .quantile(0.10)
      .reset_index()
      .rename(columns={'balance': 'balance_p10'})
)
balance_p10

,prism_consumer_id,balance_p10
0,0,52.597
1,1,403.234
2,10,83.616
3,100,907.467
4,1000,9.541
...,...,...
13004,9995,0.000
13005,9996,252.670
13006,9997,611.280
13007,9998,-862.990


In [16]:
dq_labels = (df[['prism_consumer_id', 'DQ_TARGET']].drop_duplicates())

In [17]:
model_df = balance_p10.merge(dq_labels, on='prism_consumer_id', how='inner')
model_df = model_df.dropna(subset=['DQ_TARGET'])
model_df

,prism_consumer_id,balance_p10,DQ_TARGET
0,0,52.597,0.0
1,1,403.234,0.0
2,10,83.616,0.0
3,100,907.467,0.0
4,1000,9.541,0.0
...,...,...,...
12956,995,22142.967,0.0
12966,996,0.000,0.0
12977,997,11732.731,0.0
12987,998,1340.621,0.0


In [18]:
model_df[model_df['balance_p10'] < 0]

,prism_consumer_id,balance_p10,DQ_TARGET
62,10072,-8.024,0.0
136,10168,-16.946,0.0
252,10320,-145.020,0.0
333,10422,-10.830,0.0
438,1056,-250.888,0.0
...,...,...,...
11908,8984,-4.710,0.0
11921,8996,-84.040,0.0
12165,922,-108.516,0.0
12491,952,-832.000,0.0


In [19]:
473/13009

0.03635944346221846

In [20]:
X = model_df[['balance_p10']]
y = model_df['DQ_TARGET']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

model = LogisticRegression()
model.fit(X_train, y_train)

y_prob = model.predict_proba(X_test)[:, 1]
y_pred = model.predict(X_test)

auc = roc_auc_score(y_test, y_prob)
recall = recall_score(y_test, y_pred)

print(f"ROC-AUC: {auc:.3f}")
print(f"Recall (DQ=1): {recall:.3f}")


ROC-AUC: 0.703
Recall (DQ=1): 0.000


In [21]:
y_prob.max()


0.42354919347848785

In [22]:
balance_mean = (
    df.groupby('prism_consumer_id')['balance']
      .mean()
      .reset_index()
      .rename(columns={'balance': 'balance_mean'})
)

mean_df = balance_mean.merge(dq_labels, on='prism_consumer_id')

mean_df = mean_df.dropna(subset=['DQ_TARGET'])

X = mean_df[['balance_mean']]
y = mean_df['DQ_TARGET']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

model.fit(X_train, y_train)
y_prob = model.predict_proba(X_test)[:, 1]

print("Mean balance ROC-AUC:", roc_auc_score(y_test, y_prob))


Mean balance ROC-AUC: 0.7170888661899897
